In [7]:
import akshare as ak
from typing import Optional
from typing import Any, Dict, List, Optional, Union
import pandas as pd
from typing import List, Dict, Any
import pandas as pd
import akshare as ak
from pydantic import BaseModel
from datetime import datetime


In [ ]:


# Pydantic models for request and response validation
class FinancialAbstract(BaseModel):
    reporting_period: object
    net_profit: Optional[object] = None
    net_profit_growth_rate: Optional[object] = None
    non_recurring_net_profit: Optional[object] = None
    non_recurring_net_profit_growth_rate: Optional[object] = None
    total_operating_revenue: Optional[object] = None
    total_operating_revenue_growth_rate: Optional[object] = None
    basic_earnings_per_share: Optional[object] = None
    net_asset_per_share: Optional[object] = None
    capital_reserve_fund_per_share: Optional[object] = None
    undistributed_profit_per_share: Optional[object] = None
    operating_cash_flow_per_share: Optional[object] = None
    net_profit_margin: Optional[object] = None
    gross_profit_margin: Optional[object] = None
    return_on_equity_of_roe: Optional[object] = None
    diluted_return_on_equity_of_roe: Optional[object] = None
    operating_cycle: Optional[object] = None
    inventory_turnover_ratio: Optional[object] = None
    days_inventory_outstanding: Optional[object] = None
    days_sales_outstanding: Optional[object] = None
    current_ratio: Optional[object] = None
    quick_ratio: Optional[object] = None
    conservative_quick_ratio : Optional[object] = None
    debt_to_equity_ratio: Optional[object] = None
    asset_to_liability_ratio: Optional[object] = None

def get_stock_financial_abstract(stock_code: str ,
                                 indicator: Optional[str] = '按报告期') -> list:
    
        
        result = ak.stock_financial_abstract_ths(symbol=stock_code, indicator=indicator)
        column_mapping = {
            "报告期" : "reporting_period",
            "净利润" : "net_profit",
            "净利润同比增长率" : "net_profit_growth_rate",
            "扣非净利润" : "non_recurring_net_profit",
            "扣非净利润同比增长率" : "non_recurring_net_profit_growth_rate",
            "营业总收入" : "total_operating_revenue",
            "营业总收入同比增长率" : "total_operating_revenue_growth_rate",
            "基本每股收益" : "basic_earnings_per_share",
            "每股净资产" : "net_asset_per_share",
            "每股资本公积金" : "capital_reserve_fund_per_share" ,
            "每股未分配利润" : "undistributed_profit_per_share" ,
            "每股经营现金流" : "operating_cash_flow_per_share" ,
            "销售净利率" : "net_profit_margin" ,
            "销售毛利率" : "gross_profit_margin",
            "净资产收益率" : "return_on_equity_of_roe" ,
            "净资产收益率-摊薄" : "diluted_return_on_equity_of_roe" ,
            "营业周期" : "operating_cycle" ,
            "存货周转率" : "inventory_turnover_ratio" ,
            "存货周转天数" : "days_inventory_outstanding" ,
            "应收账款周转天数" : "days_sales_outstanding" ,
            "流动比率" : "current_ratio" ,
            "速动比率" : "quick_ratio" ,
            "保守速动比率" : "conservative_quick_ratio" ,
            "产权比率" : "debt_to_equity_ratio" ,
            "资产负债率" : "asset_to_liability_ratio" ,
        }
        result.rename(columns=column_mapping, inplace=True)
        financial_abstracts = result.apply(lambda row: FinancialAbstract(**row).model_dump(), axis=1).tolist()
        print(financial_abstracts)


In [17]:
get_stock_financial_abstract(stock_code='000001')



[{'reporting_period': '1989-12-31', 'net_profit': '4302.00万', 'net_profit_growth_rate': False, 'non_recurring_net_profit': False, 'non_recurring_net_profit_growth_rate': False, 'total_operating_revenue': False, 'total_operating_revenue_growth_rate': False, 'basic_earnings_per_share': False, 'net_asset_per_share': '2.97', 'capital_reserve_fund_per_share': False, 'undistributed_profit_per_share': '1.46', 'operating_cash_flow_per_share': False, 'net_profit_margin': False, 'gross_profit_margin': None, 'return_on_equity_of_roe': False, 'diluted_return_on_equity_of_roe': '29.86%', 'operating_cycle': False, 'inventory_turnover_ratio': None, 'days_inventory_outstanding': None, 'days_sales_outstanding': False, 'current_ratio': '9.61', 'quick_ratio': '9.61', 'conservative_quick_ratio': '9.61', 'debt_to_equity_ratio': '0.28', 'asset_to_liability_ratio': '2.37%'}, {'reporting_period': '1990-12-31', 'net_profit': '7087.50万', 'net_profit_growth_rate': '64.75%', 'non_recurring_net_profit': False, 'no

In [ ]:


def get_stock_margin_detail(stock_code: str, start_date: str, end_date: str, freq: str = "D") -> List[Dict[str, Any]]:
    """
    获取中国 A 股上市公司的融资融券明细数据。

    Args:
        stock_code: 股票代码，例如 "600000"。
        start_date: 开始日期，格式为 YYYYMMDD。
        end_date: 结束日期，格式为 YYYYMMDD。
        freq: 日期间隔类型，默认为 "D"（每日）。可选值包括：
                    - "D": 每日
                    - "W": 每周
                    - "MS": 每月的第一天
                    - "ME": 每月的最后一天
                    - "Q": 每季度
                    - "Y": 每年

    Returns:
        List[Dict[str, Any]]: 返回字典的列表，每个字典表示一个交易日的融资融券概要。
        每条记录包含以下字段：
        | 字段名                 | 数据类型   | 描述                           |
        |------------------------|------------|--------------------------------|
        | trading_date           | str        | 交易日期                       |
        | target_security_code   | str        | 标的证券代码                   |
        | target_security_name   | str        | 标的证券简称                   |
        | margin_balance         | int        | 融资余额 (单位: 元)             |
        | margin_buy_amount      | int        | 融资买入额 (单位: 元)           |
        | margin_repayment       | int        | 融资偿还额 (单位: 元)           |
        | short_selling_balance  | int        | 融券余量                       |
        | short_selling_volume   | int        | 融券卖出量                     |
        | short_selling_repayment| int        | 融券偿还量                     |

    Notes:
        - 如果某一天没有数据或发生异常，会跳过该日期并继续处理后续日期。
        - 返回的数据已将中文字段名翻译为英文，便于与国际化系统集成。
    """
    # 初始化空的 DataFrame 用于存储所有数据
    all_filtered_df = pd.DataFrame()

    # 定义字段映射（中文字段名 -> 英文字段名）
    column_mapping = {
        "信用交易日期": "trading_date",
        "标的证券代码": "target_security_code",
        "标的证券简称": "target_security_name",
        "融资余额": "margin_balance",
        "融资买入额": "margin_buy_amount",
        "融资偿还额": "margin_repayment",
        "融券余量": "short_selling_balance",
        "融券卖出量": "short_selling_volume",
        "融券偿还量": "short_selling_repayment",
    }

    # 将日期范围转换为日期列表
    for date in pd.date_range(start=start_date, end=end_date, freq=freq):
        # 格式化日期为字符串
        data_date = date.strftime('%Y%m%d')

        try:
            # 获取当天的融资融券数据
            stock_margin_detail_sse_df = ak.stock_margin_detail_sse(date=data_date)

            # 检查返回的数据是否为空
            if stock_margin_detail_sse_df is None or stock_margin_detail_sse_df.empty:
                print(f"No data for date: {data_date}")
                continue  # 跳过没有数据的日期

            # 过滤证券代码为指定的 stock_code
            filtered_df = stock_margin_detail_sse_df[
                stock_margin_detail_sse_df['标的证券代码'] == stock_code
            ].copy()

            # 再次检查过滤后的数据是否为空
            if filtered_df is None or filtered_df.empty:
                print(f"No data for stock code {stock_code} on date: {data_date}")
                continue  # 跳过没有匹配数据的日期

            # 翻译字段名为英文
            filtered_df.rename(columns=column_mapping, inplace=True)

            # 将当天数据添加到总数据中
            all_filtered_df = pd.concat([all_filtered_df, filtered_df], ignore_index=True)

        except Exception as e:
            print(f"Error fetching data for date {data_date}: {e}")
            continue  # 跳过发生错误的日期

    # 将 DataFrame 转换为字典列表
    result = all_filtered_df.to_dict(orient="records")

    return result
get_stock_margin_detail(stock_code="688799", start_date='20240101', end_date='20241231',freq='MS')  

C:\Users\bigdata\AppData\Local\Temp\ipykernel_24632\1502098118.py:58: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  for date in pd.date_range(start=start_date, end=end_date, freq=freq):


Error fetching data for date 20240331: Length mismatch: Expected axis has 0 elements, new values have 13 elements
Error fetching data for date 20240630: Length mismatch: Expected axis has 0 elements, new values have 13 elements
Error fetching data for date 20240831: Length mismatch: Expected axis has 0 elements, new values have 13 elements
Error fetching data for date 20241130: Length mismatch: Expected axis has 0 elements, new values have 13 elements


[{'trading_date': '20240131',
  'target_security_code': '688799',
  'target_security_name': '华纳药厂',
  'margin_balance': 172179693,
  'margin_buy_amount': 3752975,
  'margin_repayment': 2175779,
  'short_selling_balance': 7275,
  'short_selling_volume': 1200,
  'short_selling_repayment': 1500},
 {'trading_date': '20240229',
  'target_security_code': '688799',
  'target_security_name': '华纳药厂',
  'margin_balance': 175176701,
  'margin_buy_amount': 8356871,
  'margin_repayment': 2264409,
  'short_selling_balance': 8934,
  'short_selling_volume': 700,
  'short_selling_repayment': 0},
 {'trading_date': '20240430',
  'target_security_code': '688799',
  'target_security_name': '华纳药厂',
  'margin_balance': 193543348,
  'margin_buy_amount': 11416492,
  'margin_repayment': 26807362,
  'short_selling_balance': 5918,
  'short_selling_volume': 1900,
  'short_selling_repayment': 200},
 {'trading_date': '20240531',
  'target_security_code': '688799',
  'target_security_name': '华纳药厂',
  'margin_balance'

In [14]:

class BusinessStructure(BaseModel):
    reporting_period: Optional[str]  # 报告期
    classification_direction: Optional[str] = None  # 分类方向
    classification: Optional[str] = None  # 分类
    operating_revenue: Optional[str] = None  # 营业收入（注意单位: 元）
    operating_revenue_yoy_growth: Optional[str] = None  # 营业收入-同比增长
    operating_revenue_pct_of_main: Optional[str] = None  # 营业收入-占主营收入比
    operating_cost: Optional[str] = None  # 营业成本（注意单位: 元）
    operating_cost_yoy_growth: Optional[str]  = None # 营业成本-同比增长
    operating_cost_pct_of_main: Optional[str] = None  # 营业成本-占主营成本比
    gross_profit_margin: Optional[str] = None  # 毛利率
    gross_profit_margin_yoy_growth: Optional[str] = None  # 毛利率-同比增长

# 经营业务结构
def get_stock_business_structure( stock_code: str)  -> List[Dict[str, Any]]:
        """
        获取中国A股上市公司的主要经营业务结构，用于分析公司的核心业务、产品和服务及其收入占比

        Args:
            stock_code: 股票代码，例如 "000001"。

        Returns:
            List[Dict[str, Any]]: 返回一个字典列表，一个字典表示一个报告期的报告内容。每个字典包含以下元素： 
                | reporting_period               | str  | 报告期 |
                | classification_direction       | str  | 分类方向 |
                | classification                 | str  | 分类 |
                | operating_revenue              | str  | 营业收入（注意单位: 元） |
                | operating_revenue_yoy_growth   | str  | 营业收入-同比增长 |
                | operating_revenue_pct_of_main  | str  | 营业收入-占主营收入比 |
                | operating_cost                 | str  | 营业成本（注意单位: 元） |
                | operating_cost_yoy_growth      | str  | 营业成本-同比增长 |
                | operating_cost_pct_of_main     | str  | 营业成本-占主营成本比 |
                | gross_profit_margin            | str  | 毛利率 |
                | gross_profit_margin_yoy_growth | str  | 毛利率-同比增长 |-
        """

        result = ak.stock_zygc_ym(symbol=stock_code)
        column_mapping = {
            "报告期": "reporting_period",
            "分类方向": "classification_direction",
            "分类": "classification",
            "营业收入": "operating_revenue",  # 注意单位: 元
            "营业收入-同比增长": "operating_revenue_yoy_growth",
            "营业收入-占主营收入比": "operating_revenue_pct_of_main_revenue",
            "营业成本": "operating_cost",  # 注意单位: 元
            "营业成本-同比增长": "operating_cost_yoy_growth",
            "营业成本-占主营成本比": "operating_cost_pct_of_main_cost",
            "毛利率": "gross_profit_margin",
            "毛利率-同比增长": "gross_profit_margin_yoy_growth",
        }
        result.rename(columns=column_mapping, inplace=True)
        
        stock_structure = result.apply(lambda row: BusinessStructure(**row).model_dump(), axis=1).tolist()

        return stock_structure

print(get_stock_business_structure(stock_code='000001'))


[{'reporting_period': '2023年度', 'classification_direction': '按行业分', 'classification': '零售金融业务', 'operating_revenue': '961.61亿', 'operating_revenue_yoy_growth': '-6.65%', 'operating_revenue_pct_of_main': None, 'operating_cost': '301.14亿', 'operating_cost_yoy_growth': '-6.67%', 'operating_cost_pct_of_main': None, 'gross_profit_margin': '68.68%', 'gross_profit_margin_yoy_growth': '0.01%'}, {'reporting_period': '2023年度', 'classification_direction': '按行业分', 'classification': '批发金融业务', 'operating_revenue': '535.81亿', 'operating_revenue_yoy_growth': '-15.38%', 'operating_revenue_pct_of_main': None, 'operating_cost': '168.85亿', 'operating_cost_yoy_growth': '-10.41%', 'operating_cost_pct_of_main': None, 'gross_profit_margin': '68.49%', 'gross_profit_margin_yoy_growth': '-2.49%'}, {'reporting_period': '2023年度', 'classification_direction': '按行业分', 'classification': '其他业务', 'operating_revenue': '149.57亿', 'operating_revenue_yoy_growth': '10.25%', 'operating_revenue_pct_of_main': None, 'operating_c

In [ ]:
from datetime import datetime

class NewsData(BaseModel):
    keyword: Optional[str]= None   # 关键词
    title: Optional[str]  # 新闻标题
    content: Optional[str]  # 新闻内容
    publish_time: Optional[datetime]  # 发布时间
    source: Optional[str] = None  # 文章来源
    url: Optional[str]= None   # 新闻链接
    

def stock_news( stock_code: str, start_date: Optional[ str] = None, end_date: Optional[ str] = None) -> List[Dict[str, Any]]:
        """
        Fetch the latest news articles and information related to a specific stock within a specified date range.

        Args:
            stock_code: Stock code, e.g., "000001".
            start_date: The start date of the query, formatted as 'YYYY-MM-DD'. If not provided, it defaults to one week before the current date.
            end_date: The end date of the query, formatted as 'YYYY-MM-DD'. If not provided, it defaults to the current date.

        Returns:
            List[Dict[str, Any]]: A list of dictionaries, where each dictionary represents a news article record.
            Each dictionary contains the following elements:
                | keyword       | str  | Keywords |
                | title         | str  | News Title |
                | content       | str  | News Content |
                | publish_time  | str  | Publication Time (format: YYYY-MM-DD HH:MM:SS) |
                | source        | str  | Source of the Article |
                | url           | str  | URL Link to the News Article |
        """

        column_mapping = {
            "关键词": "keyword",
            "新闻标题": "title",
            "新闻内容": "content",
            "发布时间": "publish_time",
            "文章来源": "source",
            "新闻链接": "url",
        }

        result  = ak.stock_news_em(symbol=stock_code)
        result.rename(columns=column_mapping, inplace=True)
        result['publish_time'] = pd.to_datetime(result['publish_time'], errors='coerce')
        result = result.dropna(subset=['publish_time'])

        result = result[
            (result['publish_time'] >= pd.to_datetime(start_date)) & 
            (result['publish_time'] <= pd.to_datetime(end_date))
        ]

        news = result.to_dict(orient="records")

        return news


print(stock_news(stock_code='601088', start_date='2025-01-01', end_date='2025-03-31'))

   keyword                                              title  \
21  601088               山西证券给予中国神华买入评级，运输板块部分弥补煤价下行，业绩稳健分红可期   
29  601088   开源证券给予中国神华买入评级，公司2024年报点评报告：高长协和一体化致业绩稳健，高分红行业标杆   
30  601088                                中国神华2024年度拟每股派2.26元   
31  601088                       国信证券发布中国神华研报，业绩稳健、高分红率彰显投资价值   
33  601088                            中国神华董事长吕志韧退休辞职 张长岩任授权代表   
34  601088                     3月24日46只个股获券商关注 爱美客目标涨幅达22.07%   
37  601088          分红超440亿 比例高达76%！中国神华业绩下滑 仍保持高分红“传统”|直击业绩会   
38  601088                                      中国神华：董事长吕志韧辞任   
39  601088                                      15家公司派现金额超百亿元   
40  601088                              股息率排行榜：39只股连续三年股息率超3%   
41  601088                       天风证券给予中国神华买入评级，业绩稳定性强，重视股东回报   
42  601088                              上市公司派现榜：27家每10股派现超10元   
44  601088  ETF最前线 | 华泰柏瑞上证红利ETF(510880早盘上涨0.36%，动力煤主题走强，安...   
45  601088                    中国神华：2024年度净利润586.71亿元，同比下降1.7%   
46  601088               

In [27]:
class NewsData(BaseModel):
    keyword: Optional[str]= None   # 关键词
    title: Optional[str]  # 新闻标题
    content: Optional[str]  # 新闻内容
    publish_time: Optional[datetime]  # 发布时间
    source: Optional[str] = None  # 文章来源
    url: Optional[str]= None   # 新闻链接
                                         

def stock_news_main_cx(start_date: str, end_date: str,) -> List[Dict[str, Any]]:
        """
        获取最新的财经新闻和市场动态。

        Args:
            start_date: 开始日期，格式为 YYYYMMDD。
            end_date: 结束日期，格式为 YYYYMMDD。

        Returns:
            tag	object	-
            summary	object	-
            interval_time	object	-
            pub_time	object	-
            url	object	-

        """
        column_mapping = {
            "tag": "title",
            "summary": "content",
            "pub_time": "publish_time",
            "url": "url",
        }

        result = ak.stock_news_main_cx()
        result.drop(columns=["interval_time"], inplace=True)
        result.rename(columns=column_mapping, inplace=True)

        result['publish_time'] = pd.to_datetime(result['publish_time'], errors='coerce')
        result = result.dropna(subset=['publish_time'])
        result = result[
            (result['publish_time'] >= pd.to_datetime(start_date)) & 
            (result['publish_time'] <= pd.to_datetime(end_date))
        ]

        news = result.apply(lambda row: NewsData(**row).model_dump(), axis=1).tolist()

        return news

news = stock_news_main_cx(start_date='2024-03-01', end_date='2024-04-01')


In [3]:
result=ak.stock_news_main_cx().head()
result.drop(columns=["interval_time"], inplace=True)
print(result)


                tag                                            summary  \
0  算账：美国靠加关税到底能收多少钱  市场模型最新预测显示，特朗普自1月以来已经征收以及扬言要征的所有关税将产生约3000亿美元的...   
1     美股离周期性熊市已经不远？  面临愈演愈烈的贸易战，华尔街对美股前景的预警也更为嘹亮。高盛策略团队警告，随着经济衰退风险加...   
2  新债王：市况如此波动，有人会破产  双线资本CEO，“新债王”冈拉克表示，近期剧烈的市场波动下，一些投资者或被强平，去杠杆化会扩...   
3   关税冲击下的A股调仓“小技巧”  美国关税动作的整体力度超出市场预期，市场避险情绪升温，全球风险资产随之巨震。对A股而言，短期...   
4            今日市场热点  A股早盘探底回升，三大股指翻红，资金流入军工、半导体等自主可控方向；医药板块下跌，美国总统特...   

                  pub_time                                                url  
0  2025-04-09 08:20:00.000  https://database.caixin.com/2025-04-09/1023074...  
1  2025-04-09 07:20:00.000  https://database.caixin.com/2025-04-09/1023074...  
2  2025-04-09 06:20:00.000  https://database.caixin.com/2025-04-08/1023072...  
3  2025-04-09 05:30:00.000  https://database.caixin.com/2025-04-09/1023073...  
4  2025-04-09 04:30:00.000  https://database.caixin.com/2025-04-09/1023074...  
